# MULTI-LABEL AUDIO DETECTION - SONAL AGRAWAL 200995

Importing Libraries

In [105]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, AveragePooling2D, TimeDistributed
from keras.optimizers import SGD
from tensorflow.keras.models import Sequential, model_from_json

In [107]:
path= os.getcwd()
path

'D:\\mlsp2'

In [108]:
os.chdir("D:\mlsp2")
new_path = os.getcwd()
# new_path
x_path = os.path.join(new_path , "X")
y_path = os.path.join(new_path, "Y")
val_x_path = os.path.join(new_path, "dataset", "X")
val_y_path = os.path.join(new_path, "dataset", "Y")

In [109]:
train_dataset = pd.read_csv("D:\mlsp2\\annotations.csv")
val_dataset = pd.read_csv("D:\mlsp2\\validation_data.csv")

In [110]:
train_name = train_dataset["File_name"]
train_label = train_dataset["Label"]
val_name = val_dataset["File_name"]
val_label = val_dataset["Label"]

Loading Dataset

In [111]:
temp = []
for file in train_name:
    arr = np.load(os.path.join(x_path, file))
    m,n,o = arr.shape
    arr.resize(n,o)
#     print(arr.shape)
    temp.append(arr)

In [112]:
temp[0]

array([[0.1753835 , 0.18565877, 0.18828404, ..., 0.21355054, 0.20876265,
        0.19777179],
       [0.1625914 , 0.168864  , 0.16647594, ..., 0.24709706, 0.22851093,
        0.20093745],
       [0.16749053, 0.1787626 , 0.1846619 , ..., 0.2879376 , 0.27626222,
        0.25650966],
       ...,
       [0.00619059, 0.00659246, 0.00679231, ..., 0.00702365, 0.00710513,
        0.00690014],
       [0.00418954, 0.00418414, 0.00399763, ..., 0.00372787, 0.00413942,
        0.00436432],
       [0.00302209, 0.00291839, 0.00261614, ..., 0.00219669, 0.00268963,
        0.00300021]], dtype=float32)

In [113]:
val_temp = []
for val_file in val_name:
    arr = np.load(os.path.join(val_x_path, val_file))
    m,n,o = arr.shape
    arr.resize(n,o)
#     print(arr.shape)
    val_temp.append(arr)

In [114]:
eventrolls = []
for even in train_label:
    arr = np.load(os.path.join(y_path,even))
#     m,n,o = arr.shape
#     arr.resize(n,o)
#     print(arr.shape)
    eventrolls.append(arr)

In [115]:
val_eventrolls = []
for val_even in val_label:
    arr = np.load(os.path.join(val_y_path,val_even))
#     m,n,o = arr.shape
#     arr.resize(n,o)
#     print(arr.shape)
    val_eventrolls.append(arr)

Converting eventroll to multihot-vector

In [14]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [16]:
Y = []
for eventroll in eventrolls:
    new_even = eventroll_to_multihot_vector(eventroll)
    Y.append(new_even)

In [17]:
val_Y = []
for val_eventroll in val_eventrolls:
    new_val_even = eventroll_to_multihot_vector(val_eventroll)
    val_Y.append(new_val_even)

In [18]:
X_train = np.array(temp)
Y_train = np.array(Y)
X_val = np.array(val_temp)
Y_val = np.array(val_Y)

In [19]:
X_train.shape

(10000, 64, 1000)

In [20]:
Y_train.shape

(10000, 10)

In [21]:
X_val.shape

(2000, 64, 1000)

In [22]:
Y_val.shape

(2000, 10)

In [23]:
eventrolls = np.array(eventrolls)
eventrolls.shape

(10000, 11, 1000)

In [24]:
val_eventrolls = np.array(val_eventrolls)
val_eventrolls.shape

(2000, 11, 1000)

CNN Model

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64,1000,1)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
  ])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 998, 16)       160       
                                                                 
 activation (Activation)     (None, 62, 998, 16)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 499, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 499, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 497, 32)       4640      
                                                                 
 activation_1 (Activation)   (None, 29, 497, 32)       0         
                                                        

In [27]:
from keras import backend as K
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2*((p*r)/(p+r+K.epsilon()))

In [38]:
model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy', f1, precision, recall])

In [39]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

In [40]:
history = model.fit(X_train, Y_train, validation_data= (X_val, Y_val), batch_size=128, epochs=5, verbose=1, callbacks=[es])

Epoch 1/5
79/79 [==============================] - 319s 4s/step - loss: 10.1244 - accuracy: 0.0907 - f1: 0.4403 - precision: 0.3736 - recall: 0.5399 - val_loss: 9.2113 - val_accuracy: 0.0365 - val_f1: 0.3299 - val_precision: 0.2674 - val_recall: 0.4324
Epoch 2/5
79/79 [==============================] - 291s 4s/step - loss: 8.3446 - accuracy: 0.1089 - f1: 0.4916 - precision: 0.4908 - recall: 0.4968 - val_loss: 7.5606 - val_accuracy: 0.0480 - val_f1: 0.3603 - val_precision: 0.3529 - val_recall: 0.3690
Epoch 3/5
79/79 [==============================] - 328s 4s/step - loss: 6.7826 - accuracy: 0.1125 - f1: 0.5814 - precision: 0.8885 - recall: 0.4352 - val_loss: 6.1200 - val_accuracy: 0.0480 - val_f1: 0.4331 - val_precision: 0.6230 - val_recall: 0.3326
Epoch 4/5
79/79 [==============================] - 330s 4s/step - loss: 5.4276 - accuracy: 0.1125 - f1: 0.5861 - precision: 0.9196 - recall: 0.4302 - val_loss: 4.8642 - val_accuracy: 0.0480 - val_f1: 0.4331 - val_precision: 0.6230 - val_recall

In [45]:
model.save("D:\\mlsp2\\model1.h5")

In [46]:
predictions = model.predict(X_val)

63/63 [==============================] - 10s 154ms/step


In [47]:
print(predictions)

[[0.14999041 0.15714745 0.14460084 ... 0.22683647 0.8412461  0.14179909]
 [0.15794723 0.16414279 0.15204945 ... 0.23281386 0.8337733  0.14876951]
 [0.15324678 0.15999621 0.14780228 ... 0.22926928 0.83827996 0.14419699]
 ...
 [0.15075165 0.15619217 0.14487149 ... 0.22618482 0.8412673  0.1412954 ]
 [0.14982738 0.15677194 0.1441572  ... 0.22592874 0.84134597 0.14126286]
 [0.15192829 0.1588626  0.1462853  ... 0.22746852 0.8393172  0.14279957]]


In [48]:
predictions.shape

(2000, 10)

In [71]:
result=[]
for event in predictions:
    active_events=[]
    for i in range(10):
        if ((event[i] >= 0.2).astype('float')):
            active_events.append(1)
        else:
            active_events.append(0)
    result.append(active_events)
    

In [72]:
print(result)

[[0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 0, 

In [73]:
result = np.array(result)

In [74]:
result.sum(axis=0)

array([   0,    0,    0, 2000,    0, 1627,    0, 2000, 2000,    0])

Since the data is unblanaced- assigning class weights to each class

In [100]:
class_weight = {0:0.0006671114, 1:0.0010070493, 2:0.0008865248, 3:0.0004098361 ,4:0.0007082153 , 5:0.0009124088, 6:0.0007412898, 7:0.000820344, 8: 0.0001086838, 9: 0.0009560229}

In [76]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64,1000,1)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer =tf.keras.regularizers.l1( l=0.01)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid')
  ])

In [78]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 998, 16)       160       
                                                                 
 activation_3 (Activation)   (None, 62, 998, 16)       0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 499, 16)      0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 31, 499, 16)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 497, 32)       4640      
                                                                 
 activation_4 (Activation)   (None, 29, 497, 32)       0         
                                                      

In [79]:
model2.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy', f1, precision, recall])

In [81]:
history = model2.fit(X_train, Y_train, validation_data= (X_val, Y_val), batch_size=128, epochs=20, verbose=1, callbacks=[es], class_weight=class_weight)

Epoch 1/20
79/79 [==============================] - 314s 4s/step - loss: 9.3236 - accuracy: 0.0937 - f1: 0.1985 - precision: 0.1627 - recall: 0.2547 - val_loss: 9.1477 - val_accuracy: 0.1395 - val_f1: 0.2063 - val_precision: 0.1726 - val_recall: 0.2572
Epoch 2/20
79/79 [==============================] - 292s 4s/step - loss: 7.6807 - accuracy: 0.0948 - f1: 0.2108 - precision: 0.1711 - recall: 0.2747 - val_loss: 7.5881 - val_accuracy: 0.1260 - val_f1: 0.2088 - val_precision: 0.1734 - val_recall: 0.2633
Epoch 3/20
79/79 [==============================] - 285s 4s/step - loss: 6.2006 - accuracy: 0.0970 - f1: 0.2234 - precision: 0.1805 - recall: 0.2934 - val_loss: 6.1911 - val_accuracy: 0.1205 - val_f1: 0.2131 - val_precision: 0.1757 - val_recall: 0.2716
Epoch 4/20
79/79 [==============================] - 283s 4s/step - loss: 4.8837 - accuracy: 0.1007 - f1: 0.2397 - precision: 0.1918 - recall: 0.3198 - val_loss: 4.9597 - val_accuracy: 0.1150 - val_f1: 0.2226 - val_precision: 0.1821 - val_rec

In [82]:
model2.save("D:\\mlsp2\\model2_20epochs.h5")

In [83]:
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import backend as K
def F1_score_samples(y_true, y_pred):
    return f1_score(y_true, y_pred, average='samples')
def precision_samples(y_true, y_pred):
    return precision_score(y_true, y_pred, average='samples')
def recall_samples(y_true, y_pred):
    return recall_score(y_true, y_pred, average='samples')
def F1_score_avg(y_true, y_pred):
    return f1_score(y_true, y_pred, average='micro')
def precision_avg(y_true, y_pred):
    return recall_score(y_true, y_pred, average='micro')
def recall_avg(y_true, y_pred):
    return recall_score(y_true, y_pred, average='micro')
def F1_score_class(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')
def precision_class(y_true, y_pred):
    return precision_score(y_true, y_pred, average='macro')

In [84]:
predictions2 = model2.predict(X_val)

63/63 [==============================] - 11s 169ms/step


In [85]:
print(predictions)

[[0.14999041 0.15714745 0.14460084 ... 0.22683647 0.8412461  0.14179909]
 [0.15794723 0.16414279 0.15204945 ... 0.23281386 0.8337733  0.14876951]
 [0.15324678 0.15999621 0.14780228 ... 0.22926928 0.83827996 0.14419699]
 ...
 [0.15075165 0.15619217 0.14487149 ... 0.22618482 0.8412673  0.1412954 ]
 [0.14982738 0.15677194 0.1441572  ... 0.22592874 0.84134597 0.14126286]
 [0.15192829 0.1588626  0.1462853  ... 0.22746852 0.8393172  0.14279957]]


In [97]:
result2=[]
for event in predictions2:
    active_events=[]
    for i in range(10):
        if ((event[i] >= 0.5).astype('float')):
            active_events.append(1)
        else:
            active_events.append(0)
    result2.append(active_events)
    

In [98]:
print(result2)

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 

In [116]:
test_dataset = pd.read_csv("D:\mlsp2\\test_annotations.csv")

In [117]:
test_name = test_dataset["File_name"]
# train_label = train_dataset["Label"]

In [120]:
test_path = "D:\mlsp2\\test\X"

In [121]:
test_temp = []
for file in test_name:
    arr = np.load(os.path.join(test_path, file))
    m,n,o = arr.shape
    arr.resize(n,o)
#     print(arr.shape)
    test_temp.append(arr)

In [124]:
X_test = np.array(test_temp)
X_test.shape

(2500, 64, 1000)

In [125]:
test_predictions = model.predict(X_test)

79/79 [==============================] - 17s 216ms/step


In [126]:
print(test_predictions)

[[0.1430169  0.15043662 0.13788536 ... 0.22074892 0.84787595 0.13374773]
 [0.14168555 0.148951   0.13627625 ... 0.21846952 0.8492564  0.13321717]
 [0.14338137 0.15099648 0.13786441 ... 0.22014159 0.8472021  0.13447556]
 ...
 [0.13672332 0.1443966  0.13145351 ... 0.21496029 0.85377663 0.1283348 ]
 [0.14474793 0.1522021  0.13929525 ... 0.22189508 0.84600896 0.13601278]
 [0.13622047 0.14488587 0.13112785 ... 0.21441397 0.85371    0.12755509]]


In [141]:
test_result=[]
for event in predictions:
    active_events=[]
    for i in range(10):
        if ((event[i] >= 0.2).astype('float')):
            active_events.append(1)
        else:
            active_events.append(0)
    test_result.append(active_events)
    

In [144]:
# print(test_result)

Final Conversion of predictions to csv

In [ ]:
filenames = [] # name of files
classes = []  # name of classes
events_types = {
    0: 'Alarm_bell_ringing', 
    1: 'Blender', 
    2: 'Cat', 
    3: 'Dishes', 
    4: 'Dog',
    5: 'Electric_shaver_toothbrush', 
    6: 'Frying', 
    7: 'Running_water',
    8: 'Speech', 
    9: 'Vacuum_cleaner'
}
test_name= np.array(test_name)
# name = batch["fileid"]
pred = np.array(test_result)
# pred = (pred>=0.4)
for i in range(pred.shape[0]):
    string =''
    for j in range(pred.shape[1]):
        string+=(events_types[j]+",")
        classes.append(string[:-1])
        filenames.append(test_name[i])


print(classes)
print(filenames)
df = {
    "fileid": filenames,
    "prediction": classes
}
df = pd.DataFrame(df)
df.to_csv('D:\mlsp2\\final.csv')